In [108]:
import matplotlib.pyplot as plt
import matplotlib.colors
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
from decimal import *
import imageio
import itertools
import math
import random

# Params

In [134]:
# time interval in seconds
dt = 0.001 

# Number of time steps
N_steps= 300

# Diffusion rate (if one particle type or controlled through global variable)
# D = 1

# Diffusion rate (if multiple particle types)
D_A = 1
D_B = 0.003

# Diffusion rate for particle pair
D_AB = 1

# Diffusion rate for single particle bound to DNA 
D_DNA = 0.05

# box boundaries
xlim = [-1.5, 1.5]
ylim = [-1.5, 1.5]
zlim = [-1.5, 1.5]

# number of particles
#N_particles = 100
N_A = 100
N_B = 100

# Intrinsic Reaction Rate
Ka=0.2

# Constant unbinding probability for particle-particle pairs
P_unbind = 0.1

# How far away is the particle pushed upon unbinding?
unbind_disp = 0.2

# DNA start and stop coordinates
DNA_x = [-1, 1]
DNA_y = [0, 0]
DNA_z = [0,0]

# DNA radius 
DNA_rad = 0.4

# Constant unbindg probability for particle-DNA interactions
P_unbind_DNA = 0.2

# How far away is the particle pushed upon unbinding from DNA?
DNA_unbind_disp = 0.45

# Can particles bind DNA?
can_bind_A = 0
can_bind_B = 0

Parameters used by downstream functions to determine simulation parameters: 

In [135]:
# 1 if true
DNA_present = 1

# Classes

In [136]:
class particle:
    def __init__(self, mol_type, D, x_coordinate, y_coordinate, z_coordinate, can_bind):
        self.mol_type = mol_type
        self.D = D
        self.x = x_coordinate
        self.y = y_coordinate
        self.z = z_coordinate
        self.color = int(mol_type == 'A')
        self.can_bind_DNA = can_bind
        self.DNA_bound = 0

In [137]:
class pair:
    def __init__(self, D, x_coordinate, y_coordinate, z_coordinate, can_bind,  A_idx, B_idx):
        self.A_particle_idx = A_idx
        self.B_particle_idx = B_idx
        self.D = D
        self.x = x_coordinate
        self.y = y_coordinate
        self.z = z_coordinate
        self.color = 0.5
        self.can_bind_DNA = can_bind
        self.DNA_bound = 0

# Functions

In [138]:
def get_starting_pos(mol_type, D, can_bind):
    
    """
    Generate starting position for a single particle
    The values passed in are an integer which is a string which is the type of 
    particle, and an integer which is the diffusion rate for this type of particle
    """
    
    x = np.random.uniform(low=xlim[0], high=xlim[1])
    y = np.random.uniform(low=ylim[0], high=ylim[1])
    z = np.random.uniform(low=zlim[0], high=zlim[1])
    
    return particle(mol_type,D,x,y,z,can_bind)

In [139]:
def get_starting_pos_dict(N_A, N_B):
    """
    Implements the get_starting_pos function for the appropriate number of particles. 
    Returns a dcitionary of starting positions, each of which can be used as input for the walk function.
    Takes as input a pair of integers specifying the number of particles from two classes. 
    """
    # initialize list
    pos_dict = {}
    # generate starting positions and add to list
    for i in range(N_A):
        pos_dict[i] = get_starting_pos('A', D_A, can_bind_A)
    for i in range(N_B):
        pos_dict[i+N_A] = get_starting_pos('B', D_B,can_bind_B)
        
    return pos_dict

In [140]:
def check_boundary(position, lower_bound, upper_bound):
    
    """
    Check if a particle is within the bounds of the box. If not, reflect it into the box space.
    Takes three arguments: the current position of the particle in a specific dimension (x,y,z)
    The lower boundary in that dimension
    The upper boundary in that dimension
    """
    if position < lower_bound:
        position = lower_bound - (position - lower_bound)
    elif position > upper_bound:
        position = upper_bound - (position - upper_bound)
    return position 

In [141]:
a = get_starting_pos_dict(50,6)
pairs = {}
get_DNA_binding_prob(a, pairs)

,name,x,y,z,binding_prob,class,uniform,binding


In [142]:
for i in range(1, 4):
    print(i)

1
2
3


In [143]:
def get_DNA_binding_prob(particles, pairs):
    """
    Checks that particle is not bound to DNA and is also within binding range of DNA 
    Calculates DNA binding prob and comapres to uniform random number
    Takes dictionary of particle objects and returns a pandas df with binding probabilities. 
    """
    # Get a list of unbound particles that can bind DNA
    unbound_particles=[i for (i,v) in particles.items() if (v.DNA_bound == 0 and v.can_bind_DNA == 1)]
    
    unbound_pairs=[i for(i,v) in pairs.items() if (v.DNA_bound == 0 and v.can_bind_DNA == 1)]
    
    # Get list of particles within range of DNA 
    within_range = []
    for part in unbound_particles:
        if abs(particles[part].y) < DNA_rad and abs(particles[part].z) < DNA_rad:
            within_range.append([part, particles[part].x, particles[part].y, particles[part].z])
    
    # Make pandas df
    DNA_binding_df = pd.DataFrame(within_range, columns = ['name', 'x', 'y', 'z'])

    # Calculate Binding Probability and Compare to Uniform Rand Number
    DNA_binding_df["binding_prob"] = 0.2
    DNA_binding_df["class"] = 'particle'
    DNA_binding_df["uniform"] = np.random.uniform(0,1, DNA_binding_df.shape[0])
    DNA_binding_df["binding"] = DNA_binding_df["binding_prob"] > DNA_binding_df["uniform"]
    DNA_binding_df["binding"].astype(int)
    
    # Get list of pairs within range of DNA 
    within_range_pairs = []
    for pair in unbound_pairs:
        if abs(pairs[pair].y) < DNA_rad and abs(pairs[pair].z) < DNA_rad:
            within_range_pairs.append([[pair, pairs[pair].A_particle_idx, pairs[pair].B_particle_idx], pairs[pair].x, pairs[pair].y, pairs[pair].z])
    
    # Make pandas df
    DNA_binding_df_pairs = pd.DataFrame(within_range_pairs, columns = ['name', 'x', 'y', 'z'])

    # Calculate Binding Probability and Compare to Uniform Rand Number
    DNA_binding_df_pairs["binding_prob"] = 0.2
    DNA_binding_df_pairs["class"] = 'pair'
    DNA_binding_df_pairs["uniform"] = np.random.uniform(0,1, DNA_binding_df_pairs.shape[0])
    DNA_binding_df_pairs["binding"] = DNA_binding_df_pairs["binding_prob"] > DNA_binding_df_pairs["uniform"]
    DNA_binding_df_pairs["binding"].astype(int)
    
    return pd.concat([DNA_binding_df, DNA_binding_df_pairs])

In [144]:
def bind_DNA(particles, pairs, DNA_binding_df): 
    """
    Takes as input a dictionary of particle objects and a pandas df as created by get_DNA_binding_prob 
    Uses binding probabilities to modify particle objects
    Outputs a set of modified particle objects incorporating binding and also a list of new binding events
    """
    # List of DNA_bound particles and pairs
    DNA_bound_particles = []
    DNA_bound_pairs = []
    
    # Get only the particles that bind
    DNA_binding_df_true = DNA_binding_df[DNA_binding_df["binding"] == True]
    
    # Loop through particles that bind and modify:
    for i in range(DNA_binding_df_true.shape[0]):
        if DNA_binding_df_true.iloc[i]["class"] == 'particle':                              
            p_name=DNA_binding_df_true.iloc[i]["name"]

            particles[p_name].DNA_bound = 1
            #particles[p_name].color = 1
            particles[p_name].D = D_DNA

            DNA_bound_particles.append(p_name)
        else:
            p_name=DNA_binding_df_true.iloc[i]["name"][0]  
            
            pairs[p_name].DNA_bound = 1
            pairs[p_name].D = D_DNA
                                         
            DNA_bound_pairs.append(p_name)
    
    return particles, pairs, DNA_bound_particles, DNA_bound_pairs

In [145]:
def unbind_DNA(particles, pairs, DNA_bound_list, DNA_bound_pairs):
    """
    Takes as input a dictionary of particle objects and a list of DNA-bound particles as generated by bind_DNA() 
    """
    # List of possible particle-DNA unbinding events
    l =[]
    # Loop through all pairs of bound molecules
    for particle in DNA_bound_list:
        # Get random number from uniform distribution
        rand_num = np.random.uniform(0,1)
        # Update list of possible unbinding events
        l.append([particle, P_unbind_DNA, rand_num])
        if P_unbind_DNA > rand_num: 
            # Remove particle from list of DNA-bound molecules 
            DNA_bound_list.remove(particle)
            
            # Update bound state of particle object
            particles[particle].DNA_bound = 0
            
            #particles[particle].color = int(particles[particle].mol_type == 'A')
            
            # displace particles
            particles[particle].x = float(check_boundary(particles[particle].x + random.choice([-1,1])*DNA_unbind_disp, xlim[0], xlim[1]))
            particles[particle].y = float(check_boundary(particles[particle].y + random.choice([-1,1])*DNA_unbind_disp, ylim[0], ylim[1]))
            particles[particle].z = float(check_boundary(particles[particle].z + random.choice([-1,1])*DNA_unbind_disp, zlim[0], zlim[1]))
            
            # Update diffusion constant
            if particles[particle].mol_type == 'A':
                particles[particle].D = D_A
            else:
                particles[particle].D = D_B
    
    df_unbind_DNA_p1=pd.DataFrame(l, columns = ['name', 'P_unbind_DNA', 'rand_num'])
    df_unbind_DNA_p1["class"] = 'particle'
    
    l_pairs = []
    for pair in DNA_bound_pairs:
        # Get random number from uniform distribution
        rand_num = np.random.uniform(0,1)
        # Update list of possible unbinding events
        l_pairs.append([[pair, pairs[pair].A_particle_idx, pairs[pair].B_particle_idx], P_unbind_DNA, rand_num])
        if P_unbind_DNA > rand_num: 
            # Remove particle from list of DNA-bound molecules 
            DNA_bound_pairs.remove(pair)
            
            # Update bound state of pairs object
            pairs[pair].DNA_bound = 0
        
            # displace pair
            pairs[pair].x = float(check_boundary(pairs[pair].x + random.choice([-1,1])*DNA_unbind_disp, xlim[0], xlim[1]))
            pairs[pair].y = float(check_boundary(pairs[pair].y + random.choice([-1,1])*DNA_unbind_disp, ylim[0], ylim[1]))
            pairs[pair].z = float(check_boundary(pairs[pair].z + random.choice([-1,1])*DNA_unbind_disp, zlim[0], zlim[1]))
       
    df_unbind_DNA_p2=pd.DataFrame(l_pairs, columns = ['name', 'P_unbind_DNA', 'rand_num'])
    df_unbind_DNA_p2["class"] = 'pair'
    
    df_unbind_DNA = pd.concat([df_unbind_DNA_p1, df_unbind_DNA_p2])
        
    return particles, pairs, df_unbind_DNA , DNA_bound_list, DNA_bound_pairs

In [146]:
def radius(p1, p2):
    """
    Takes two points and returns the distance between them. 
    Each point is a particle-class object.
    This is pretty trivial, but also doing this as a function makes the code for find_radii much more legible. 
    """
    return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2 + (p1.z - p2.z)**2)

In [147]:
def find_radii(positions):
    """
    Takes a dictionary of positions and returns a list of all the pairwise distances between elements
    List is formatted as [[x1,y1,z1], [x2,y2,z2], etc.]
    """
    # Find radii between all unbound positions
    l=[]
    for p1, p2 in itertools.combinations(positions, 2):
        if positions[p1].mol_type != positions[p2].mol_type:
            l.append([p1, p2, radius(positions[p1], positions[p2])])
    return pd.DataFrame(l, columns = ['p1_idx', 'p2_idx', 'distance'])

In [148]:
def calc_binding_prob(df):
    """
    This takes a pandas data frame as generated by find_radii and adds a column with binding probability.
    Also adds a column with a random number from 0 to 1 pulled from a uniform distribution. 
    Returns a data frame.
    Note that the binding probability is calculated using an arbitrary equation. 
    """
    # Calcuolate binding probability using arbitrary equation
    df["binding_prob"] = 1/(1000*(df["distance"]+0.94)**2) #6)
    #df["binding_prob"] = 1/((df["distance"]+0.94)**2) #6)
    #df["binding_prob"] = 0.1
    # Get random number from uniform distribution
    df["uniform"] = np.random.uniform(0,1, df.shape[0])
    # Compare binding prob and random number to determine binding
    df["binding"] = df["binding_prob"] > df["uniform"]
    df["binding"].astype(int)
    return df

In [149]:
def make_pair(particles, part_A_name, part_B_name):
    
    """
    Instantiate particle pair based on two binding particle objects
    Takes a list of particle objects and the indeces of two specific particles that are binding 
    """
    x = np.mean([particles[part_A_name].x, particles[part_B_name].x])
    y = np.mean([particles[part_A_name].y, particles[part_B_name].y])
    z = np.mean([particles[part_A_name].z, particles[part_B_name].z])
    
    return pair(D_AB,x,y,z,1,part_A_name,part_B_name)

In [150]:
def bind(particles, pairs, df):
    """
    Takes a dictionary of particles, a dictionary of pairs, and a dataframe generated by calc_binding_prob
    Returns updated dictionary of particles with binding particles removed
    Returns updated dictionary of pairs with new binding pairs added
    """
    # Get only the particles that bind
    df = df[df["binding"] == True]
    # sort binding by probability
    df=df.sort_values(by = "binding_prob", ascending=False)
    # initialize list of binding partners and also a list of every unique molecule seen 
    binding_partners = []
    bound_particles = []
    # loop through sorted list and only extract binding pairs with molecules that have not been seen yet
    for i in range(df.shape[0]):
        if df.iloc[i]["p1_idx"] not in bound_particles and df.iloc[i]["p2_idx"] not in bound_particles:
            binding_partners.append([df.iloc[i]["p1_idx"], df.iloc[i]["p2_idx"]])
            bound_particles.append(df.iloc[i]["p1_idx"])
            bound_particles.append(df.iloc[i]["p2_idx"])
    
    # FOR DEBUG
    #print(binding_partners)
            
    # Update the dictionary of particles to reflect binding
    for i in binding_partners:  
        ## Make a new pair
        # Get highest key name from pairs dictionary
        if len(pairs) > 0:
            key_name = max(pairs.keys()) + 1
        else: 
            key_name = 0
        # Make new particle pair
        pairs[key_name]=(make_pair(particles, i[0], i[1]))
        
        ## Update particles dictionary to exclude newly bound particles 
        del particles[i[0]]
        del particles[i[1]]
            
    return particles, pairs

In [151]:
def unbind(particles, pairs):
    """
    Loops through a dictionary of bound particle pairs and determins if that list of particles will unbind. 
    Takes as input a dictionary of particle objects and a dictionary of bound particle pair objects
    Returns updated dictionary of particle objects
    Returns data frame of unbinding events
    Returns updated dictionary of bound molecules
    """
    # List of possible unbinding events
    l =[]
    # List of entries to remove from dictionary
    to_remove = []
    # Loop through all pairs of bound molecules
    for (i,v) in pairs.items():
        # Get random number from uniform distribution
        rand_num = np.random.uniform(0,1)
        # Update list of possible unbinding events
        l.append([v.A_particle_idx, v.B_particle_idx, P_unbind, rand_num])
        # Compare to undbinding probability 
        if P_unbind > rand_num: 
            ## Add particles back to particle list  
            for idx in [v.A_particle_idx, v.B_particle_idx]:
                if idx < N_A: 
                    particles[idx] = particle('A', D_A, v.x, v.y, v.z, can_bind_A)
                else:
                    new_x = float(check_boundary(v.x + random.choice([-1,1])*unbind_disp, xlim[0], xlim[1]))
                    new_y = float(check_boundary(v.y + random.choice([-1,1])*unbind_disp, ylim[0], ylim[1]))
                    new_z = float(check_boundary(v.z + random.choice([-1,1])*unbind_disp, zlim[0], zlim[1]))
                    particles[idx] = particle('B', D_B, new_x, new_y, new_z, can_bind_B)
            
            ## Add key to list of particle pairs that are dissovling (ie to remove from dict)
            to_remove.append(i)
    
    # Remove keys from dictionary
    for key in to_remove:
        del pairs[key]
    
    return particles, pd.DataFrame(l, columns = ['p1_idx', 'p2_idx', 'P_unbind', 'rand_num']), pairs

In [152]:
def walk(particles, pairs):
    """ 
    Takes as argument a dictionary particle objects. 
    Draws a displacements from a Gaussian distribution.
    Checks if new position is within boundaries and adjusts if not.
    Returns updated dictionarty particle objects with new positions.
    """
    # Loop through all particles
    for i in particles.keys():
        # Move DNA bound molecules
        if particles[i].DNA_bound == 1:
            x_disp = np.random.randn(1)*np.sqrt(2*dt*particles[i].D)
            x_pos = particles[i].x + x_disp
            # Check positions
            particles[i].x = float(check_boundary(x_pos, xlim[0], xlim[1]))
            particles[i].y = 0
            particles[i].z = 0
        
    
        # If not DNA bound, get positions from gaussian distribution
        else:   
            x_disp = np.random.randn(1)*np.sqrt(2*dt*particles[i].D)
            y_disp = np.random.randn(1)*np.sqrt(2*dt*particles[i].D)
            z_disp = np.random.randn(1)*np.sqrt(2*dt*particles[i].D)

            x_pos = particles[i].x + x_disp
            y_pos = particles[i].y + y_disp
            z_pos = particles[i].z + z_disp

            # Check positions
            particles[i].x = float(check_boundary(x_pos, DNA_x[0], DNA_x[1]))
            particles[i].y = float(check_boundary(y_pos, ylim[0], ylim[1]))
            particles[i].z = float(check_boundary(z_pos, zlim[0], zlim[1]))
            
     # Loop through all particles
    for i in pairs.keys():
        # Move DNA bound molecules
        if pairs[i].DNA_bound == 1:
            x_disp = np.random.randn(1)*np.sqrt(2*dt*pairs[i].D)
            x_pos = pairs[i].x + x_disp
            # Check positions
            pairs[i].x = float(check_boundary(x_pos, xlim[0], xlim[1]))
            pairs[i].y = 0
            pairs[i].z = 0
        
    
        # If not DNA bound, get positions from gaussian distribution
        else:   
            x_disp = np.random.randn(1)*np.sqrt(2*dt*pairs[i].D)
            y_disp = np.random.randn(1)*np.sqrt(2*dt*pairs[i].D)
            z_disp = np.random.randn(1)*np.sqrt(2*dt*pairs[i].D)

            x_pos = pairs[i].x + x_disp
            y_pos = pairs[i].y + y_disp
            z_pos = pairs[i].z + z_disp

            # Check positions
            pairs[i].x = float(check_boundary(x_pos, DNA_x[0], DNA_x[1]))
            pairs[i].y = float(check_boundary(y_pos, ylim[0], ylim[1]))
            pairs[i].z = float(check_boundary(z_pos, zlim[0], zlim[1]))
            
        
    return particles, pairs

In [153]:
def plot_trajectory(particles, pairs, step_number):
    """ 
    Takes as input a dictionary of particle objects at their current positions.
    Takes as input an integer representing the time step. 
    Plots all particles 
    """
    # initialize 3D plot 
    fig = plt.figure(figsize=(12,8), dpi= 100)
    ax = plt.axes(projection='3d')
    
    # For this time point, extract all of the coordinates to be plotted 
    plt_x = []
    plt_y = []
    plt_z = []
    color = []
    
    for (i, v) in particles.items():
        plt_x.append(v.x)
        plt_y.append(v.y)
        plt_z.append(v.z)
        color.append(v.color)
        
    for (i, v) in pairs.items():
        plt_x.append(v.x)
        plt_y.append(v.y)
        plt_z.append(v.z)
        color.append(v.color)
        
    cmap = plt.cm.rainbow
    norm = matplotlib.colors.Normalize(vmin=0, vmax=1)  
    
    ax.scatter3D(plt_x, plt_y, plt_z, c=cmap(norm(color)), cmap='jet')
    if DNA_present == 1: 
        ax.plot(DNA_x, DNA_y, DNA_z, 'black', linestyle=':', marker='', linewidth=2)
        
    plt.xlim([-1.5, 1.5])
    plt.ylim([-1.5, 1.5])
    ax.set_zlim([-1.5, 1.5])
    
    # create filename and save figure
    fname="out/fig_"+str(step_number)
    plt.savefig(fname)
    plt.close()
    
    return fname+".png"

In [154]:
def initialize_sim(N_A, N_B):
    ## Initializing walk
    # Generate an empty list used to store filenames of images to be stiched into a filme
    fnames = []
    # Get starting positions for the correct number of particles.
    particles = get_starting_pos_dict(N_A,N_B)
    pairs = {}
    # Store starting positions in a dataframe
    df_pos = pd.DataFrame([vars(s) for s in particles.values()],  columns=['x', 'y', 'z'])
    df_pos.insert(0, "name", pd.DataFrame(particles.keys()))
    df_pos["time_step"] = 0
    df_pos.to_csv('pos.csv', index=False)
    # Store starting radii in a dataframe 
    df_rad=find_radii(particles)
    # Calculate binding probabilities
    df_rad=calc_binding_prob(df_rad)
    df_rad["time_step"] = 0
    df_rad.to_csv('rad.csv', index=False)
    
    # Initialize empty csv for particle-particle unbinding 
    df_unbind=pd.DataFrame(columns=['p1_idx', 'p2_idx', 'P_unbind', 'rand_num', 'time_step'])
    df_unbind.to_csv('unbind.csv', mode = 'w', header=True, index=False)
    
    # Initialize empty csv for particle-DNA unbinding 
    if DNA_present == 1:
        df_unbind_DNA=pd.DataFrame(columns=['name', 'P_unbind_DNA', 'rand_num', 'class', 'time_step'])
        df_unbind_DNA.to_csv('unbind_DNA.csv', mode = 'w', header=True, index=False)
        
    ## Bind molecules and generate list of bound molecules
    [particles, pairs] = bind(particles, pairs, df_rad)
    
    ## Bind DNA 
    if DNA_present == 1: 
        # Calculate Binding Probability and write to csv
        DNA_df = get_DNA_binding_prob(particles, pairs)
        DNA_df["time_step"] = 0
        DNA_df.to_csv('DNA_bind.csv', mode = 'w', header=True, index=False)
        
        # Update particle objects and also create list of DNA bound particles
        [particles, pairs, DNA_bound_particles, DNA_bound_pairs] = bind_DNA(particles, pairs, DNA_df)
    
    ## Plot the initial position
    plot_trajectory(particles,pairs, 0)

    return particles, pairs, fnames, DNA_bound_particles, DNA_bound_pairs

In [155]:
def run_sim(N_steps, particles, pairs, fnames, DNA_bound_particles, DNA_bound_pairs):
    for i in range(1, N_steps):
        # Update position
        [particles, pairs]=walk(particles, pairs)
        
        ## Unbind DNA 
        if DNA_present == 1: 
            [particles, pairs, df_unbind_DNA, DNA_bound_particles, DNA_bound_pairs] = unbind_DNA(particles, pairs, DNA_bound_particles, DNA_bound_pairs)
            df_unbind_DNA["time_step"] = i
            df_unbind_DNA.to_csv('unbind_DNA.csv', mode = 'a', header=False, index=False)
            
        ## Unbind molecules 
        [particles, df_unbind, pairs] = unbind(particles, pairs)
        # Write unbound list
        df_unbind["time_step"] = i      
        df_unbind.to_csv('unbind.csv', mode = 'a', header=False, index=False)
        
        ## Updating df_pos
        # Store updated positions into dataframe
        df_pos = pd.DataFrame([vars(s) for s in particles.values()],  columns=['x', 'y', 'z'])
        if len(df_pos) > 0:
            df_pos.insert(0, "name", pd.DataFrame(particles.keys()))
            # Add time step
            df_pos["time_step"] = i 
            # Write to file, appending
            df_pos.to_csv('pos.csv', mode = 'a', header=False, index=False)
        
        ## Updating df_rad
        # Calculate radis between all particles
        df_rad = find_radii(particles)
        # Calculate binding probabilities 
        df_rad=calc_binding_prob(df_rad)
        # Add time step
        df_rad["time_step"] = i 
        # Write to csv
        df_rad.to_csv('rad.csv', mode = 'a', header=False, index=False)  
        
        ## Bind molecules 
        [particles, pairs] = bind(particles, pairs, df_rad)
        
        ## Bind DNA 
        if DNA_present == 1: 
            # Calculate Binding Probability and write to csv 
            DNA_df = get_DNA_binding_prob(particles, pairs)
            DNA_df["time_step"] = i
            DNA_df.to_csv('DNA_bind.csv', mode = 'a', header=False, index=False)
            [particles, pairs, DNA_bound_particles, DNA_bound_pairs] = bind_DNA(particles, pairs, DNA_df)
            
            # Update list of bound molecules
            DNA_bound_particles = DNA_bound_particles + DNA_bound_particles
        
        ## Plot updated position and add file name to the list of filenames
        fnames.append(plot_trajectory(particles, pairs, i))
    
    # convert images to movie
    frames = []  
    print(fnames)
    for img in fnames:
        frames.append(imageio.imread(img))
    imageio.mimwrite('mv.mp4', frames, '.mp4', fps=30)
    
    return df_pos, df_rad

In [156]:
[particles, pairs, fnames, DNA_bound_particles, DNA_bound_pairs] = initialize_sim(N_A, N_B)
[x, y] = run_sim(N_steps, particles, pairs, fnames, DNA_bound_particles, DNA_bound_pairs)

['out/fig_1.png', 'out/fig_2.png', 'out/fig_3.png', 'out/fig_4.png', 'out/fig_5.png', 'out/fig_6.png', 'out/fig_7.png', 'out/fig_8.png', 'out/fig_9.png', 'out/fig_10.png', 'out/fig_11.png', 'out/fig_12.png', 'out/fig_13.png', 'out/fig_14.png', 'out/fig_15.png', 'out/fig_16.png', 'out/fig_17.png', 'out/fig_18.png', 'out/fig_19.png', 'out/fig_20.png', 'out/fig_21.png', 'out/fig_22.png', 'out/fig_23.png', 'out/fig_24.png', 'out/fig_25.png', 'out/fig_26.png', 'out/fig_27.png', 'out/fig_28.png', 'out/fig_29.png', 'out/fig_30.png', 'out/fig_31.png', 'out/fig_32.png', 'out/fig_33.png', 'out/fig_34.png', 'out/fig_35.png', 'out/fig_36.png', 'out/fig_37.png', 'out/fig_38.png', 'out/fig_39.png', 'out/fig_40.png', 'out/fig_41.png', 'out/fig_42.png', 'out/fig_43.png', 'out/fig_44.png', 'out/fig_45.png', 'out/fig_46.png', 'out/fig_47.png', 'out/fig_48.png', 'out/fig_49.png', 'out/fig_50.png', 'out/fig_51.png', 'out/fig_52.png', 'out/fig_53.png', 'out/fig_54.png', 'out/fig_55.png', 'out/fig_56.png', 

In [306]:
a[1].x

array([0.72476429])

In [303]:
x

,name,x,y,z,time_step
0,0,[0.7386856311477143],[0.5863163615274724],[-0.016709947176582113],9
1,1,[-0.33800883597710757],[1.1422218836595728],[1.041976007102884],9
2,2,[-0.8122061215750805],[0.9159107571292823],[-1.199789414207695],9
3,3,[0.14363167992132142],[-0.05675028582447683],[1.2982862467534886],9
4,4,[-0.828692844893156],[-0.9605329722694069],[-0.6158179917673562],9
5,5,[-0.2797229780200664],[0.6596545847826466],[1.0666962025645577],9
6,6,[-0.6000723538327303],[0.5874431552499805],[1.090129251236468],9
7,7,[-0.5722621790334768],[-0.1855528910114862],[0.4678797749855501],9
8,8,[0.0893831700056546],[1.1672721060261924],[0.8308766575531542],9
9,9,[-0.1652526136666166],[0.6875719684853696],[-0.6031015327785408],9


In [32]:
a = get_starting_pos_dict(100,100)
walk(a)

{0: <__main__.particle at 0x7fd61e4da460>,
 1: <__main__.particle at 0x7fd61e51cf70>,
 2: <__main__.particle at 0x7fd61e51cfd0>,
 3: <__main__.particle at 0x7fd61e51ceb0>,
 4: <__main__.particle at 0x7fd61e51ce80>,
 5: <__main__.particle at 0x7fd61e51cdc0>,
 6: <__main__.particle at 0x7fd61e51ce20>,
 7: <__main__.particle at 0x7fd61e51c9d0>,
 8: <__main__.particle at 0x7fd61e51cd60>,
 9: <__main__.particle at 0x7fd61e51ccd0>,
 10: <__main__.particle at 0x7fd61e51cd00>,
 11: <__main__.particle at 0x7fd61e51cc70>,
 12: <__main__.particle at 0x7fd61e51cee0>,
 13: <__main__.particle at 0x7fd61e51cfa0>,
 14: <__main__.particle at 0x7fd61e51c970>,
 15: <__main__.particle at 0x7fd61e51cbe0>,
 16: <__main__.particle at 0x7fd61e51cd90>,
 17: <__main__.particle at 0x7fd61e51cc10>,
 18: <__main__.particle at 0x7fd61e51cd30>,
 19: <__main__.particle at 0x7fd61e51cf10>,
 20: <__main__.particle at 0x7fd61e51c880>,
 21: <__main__.particle at 0x7fd61e51cf40>,
 22: <__main__.particle at 0x7fd61e528040>

In [33]:
a = get_starting_pos_dict(100,100)
b = get_DNA_binding_prob(a)
pairs = {}
bp = calc_binding_prob(find_radii(a))
[bind1, bind2] = bind(a, pairs, bp)

[[81, 125], [95, 175], [35, 196]]


In [ ]:
unbind(a, bind2)

In [110]:
[e,f,g] = unbind_DNA(c,d)

In [119]:
e[80].x

-0.6744535084062074

In [95]:
[c,d] = bind_DNA(a, b)

In [99]:
c[66].D

0.05

In [100]:
d

[66, 80]

In [71]:
a = get_starting_pos_dict(100,100)
b = get_DNA_binding_prob(a)

In [35]:
get_DNA_binding_prob(a)

In [15]:
a = get_starting_pos_dict(N_A, N_B)

In [17]:
a[1].x

-0.4851210517106632